<a href="https://colab.research.google.com/github/desstaw/DataPrivacy_SimulatedAnnealing/blob/main/Main_K_Anonymity_Simulated_Annealing_Utility_Diabetes70K_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



## Import the data and briefly explore it again

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import roc_auc_score, roc_curve
import matplotlib.pyplot as plt

import random
import math

In [ ]:
url = "/content/drive/MyDrive/Colab Notebooks/Diabetes/arx_gen_diabetes_70K.csv"
df = pd.read_csv(url)

In [ ]:
df = df.rename(columns={'L4_num_medications': 'L3_num_medications'})

In [ ]:
df.columns

Index(['L4_race', 'gender', 'L3_age', 'L3_admission_type_id', 'readmitted',
       'discharge_disposition_id', 'admission_source_id',
       'L4_time_in_hospital', 'num_lab_procedures', 'num_procedures',
       'num_medications', 'number_outpatient', 'L3_number_emergency',
       'number_inpatient', 'number_diagnoses', 'max_glu_serum', 'A1Cresult',
       'metformin', 'repaglinide', 'nateglinide', 'glimepiride', 'glipizide',
       'glyburide', 'pioglitazone', 'rosiglitazone', 'insulin', 'change',
       'diabetesMed', 'service_utilization', 'numchange', 'L3_diagnosis',
       'nummed', 'L1_race', 'L2_race', 'L3_race', 'L1_time_in_hospital',
       'L2_time_in_hospital', 'L3_time_in_hospital', 'lower_bound', 'L1_age',
       'L2_age', 'L1_number_emergency', 'L2_number_emergency', 'L1_diagnosis',
       'L2_diagnosis', 'L2_admission_type_id', 'L1_admission_type_id'],
      dtype='object')

In [ ]:

# Define the new order of column names
new_order = ['gender', 'readmitted', 'discharge_disposition_id', 'admission_source_id', 'num_lab_procedures', 'number_outpatient', 'number_inpatient', 'number_diagnoses',
             'max_glu_serum', 'A1Cresult', 'metformin', 'repaglinide', 'nateglinide', 'glimepiride', 'glipizide', 'glyburide', 'pioglitazone', 'rosiglitazone', 'insulin', 'change',
             'diabetesMed', 'service_utilization', 'numchange',
             'L1_age', 'L2_age', 'L3_age', #y
             'L1_race', 'L2_race', 'L3_race', 'L4_race', #y
             'L1_admission_type_id', 'L2_admission_type_id', 'L3_admission_type_id',
             'L1_time_in_hospital', 'L2_time_in_hospital', 'L3_time_in_hospital', 'L4_time_in_hospital', #y
             'num_procedures',
             'num_medications',
             'L1_diagnosis', 'L2_diagnosis', 'L3_diagnosis',
             'L1_number_emergency', 'L2_number_emergency', 'L3_number_emergency',
             'readmitted']

# Reorder the columns
df = df[new_order]


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74647 entries, 0 to 74646
Data columns (total 46 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   gender                    74647 non-null  object
 1   readmitted                74647 non-null  int64 
 2   discharge_disposition_id  74647 non-null  int64 
 3   admission_source_id       74647 non-null  int64 
 4   num_lab_procedures        74647 non-null  int64 
 5   number_outpatient         74647 non-null  int64 
 6   number_inpatient          74647 non-null  int64 
 7   number_diagnoses          74647 non-null  int64 
 8   max_glu_serum             74647 non-null  object
 9   A1Cresult                 74647 non-null  object
 10  metformin                 74647 non-null  object
 11  repaglinide               74647 non-null  object
 12  nateglinide               74647 non-null  object
 13  glimepiride               74647 non-null  object
 14  glipizide             

In [ ]:
df = df.loc[:, ~df.columns.duplicated(keep='first')]



## Necessary pre-processing for CatBoost

In [ ]:
df ['readmitted'] = df ['readmitted'].astype(str)

df ['L1_age'] = df ['L1_age'].astype(str)
df ['L2_age'] = df ['L2_age'].astype(str)
df ['L3_age'] = df ['L3_age'].astype(str)


df ['L1_admission_type_id'] = df ['L1_admission_type_id'].astype(str)
df ['L2_admission_type_id'] = df ['L2_admission_type_id'].astype(str)
df ['L3_admission_type_id'] = df ['L3_admission_type_id'].astype(str)


df ['L4_time_in_hospital'] = df ['L4_time_in_hospital'].astype(str)

df ['L3_number_emergency'] = df ['L3_number_emergency'].astype(str)


<ipython-input-9-ce5c308d8c3e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df ['readmitted'] = df ['readmitted'].astype(str)
<ipython-input-9-ce5c308d8c3e>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df ['L1_age'] = df ['L1_age'].astype(str)
<ipython-input-9-ce5c308d8c3e>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/s

Visual distribution of categorical attributes

In [ ]:
'''
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Define the list of categorical columns to visualize
#cat_cols = ['sex', 'L7_age', 'L5_education', 'L3_occupation', 'L3_marital_status', 'L3_workclass', 'L3_native_country', 'L2_race', 'salary-class']
cat_cols = df.select_dtypes(include=['object']).columns

# Determine the number of subplots based on the number of categorical columns
num_subplots = len(cat_cols)

# Set up the subplots
fig, axes = plt.subplots(num_subplots, 1, figsize=(10, 4 * num_subplots))

# Loop through each categorical column and create a countplot in the corresponding subplot
for i, column in enumerate(cat_cols):
    sns.countplot(data=df, x=column, ax=axes[i])
    axes[i].set_xticklabels(axes[i].get_xticklabels(), rotation=30)  # Rotate x-axis labels for better readability
    axes[i].set_title(f'Distribution of {column}')
    axes[i].set_xlabel(None)  # Remove x-axis label for better layout

# Adjust the layout
plt.tight_layout()

# Show the plots
plt.show()
'''

"\nimport pandas as pd\nimport matplotlib.pyplot as plt\nimport seaborn as sns\n\n# Define the list of categorical columns to visualize\n#cat_cols = ['sex', 'L7_age', 'L5_education', 'L3_occupation', 'L3_marital_status', 'L3_workclass', 'L3_native_country', 'L2_race', 'salary-class']\ncat_cols = df.select_dtypes(include=['object']).columns\n\n# Determine the number of subplots based on the number of categorical columns\nnum_subplots = len(cat_cols)\n\n# Set up the subplots\nfig, axes = plt.subplots(num_subplots, 1, figsize=(10, 4 * num_subplots))\n\n# Loop through each categorical column and create a countplot in the corresponding subplot\nfor i, column in enumerate(cat_cols):\n    sns.countplot(data=df, x=column, ax=axes[i])\n    axes[i].set_xticklabels(axes[i].get_xticklabels(), rotation=30)  # Rotate x-axis labels for better readability\n    axes[i].set_title(f'Distribution of {column}')\n    axes[i].set_xlabel(None)  # Remove x-axis label for better layout\n\n# Adjust the layout\np

In [ ]:
#df = df.drop('ID', axis=1)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74647 entries, 0 to 74646
Data columns (total 45 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   gender                    74647 non-null  object
 1   readmitted                74647 non-null  object
 2   discharge_disposition_id  74647 non-null  int64 
 3   admission_source_id       74647 non-null  int64 
 4   num_lab_procedures        74647 non-null  int64 
 5   number_outpatient         74647 non-null  int64 
 6   number_inpatient          74647 non-null  int64 
 7   number_diagnoses          74647 non-null  int64 
 8   max_glu_serum             74647 non-null  object
 9   A1Cresult                 74647 non-null  object
 10  metformin                 74647 non-null  object
 11  repaglinide               74647 non-null  object
 12  nateglinide               74647 non-null  object
 13  glimepiride               74647 non-null  object
 14  glipizide             

In [ ]:
#Since the catboost model cannot read columns with interval values which are originally intended to be categories in the form of interval strings
def generate_interval_mappings(df, interval_columns):
    interval_mapping = {}

    for column in interval_columns:
        unique_intervals = df[column].unique()
        for interval in unique_intervals:
            if isinstance(interval, tuple):  # Check if the interval is a tuple
                mapping_key = f"{column}_{interval[0]}_{interval[1]}"
                interval_mapping[interval] = mapping_key
                print('interval mapping', interval_mapping)

    return interval_mapping

# List of columns with interval values
interval_columns = df[['L1_age', 'L2_age', 'L3_age', #y
             'L1_race', 'L2_race', 'L3_race', 'L4_race', #y
             'L1_admission_type_id', 'L2_admission_type_id', 'L3_admission_type_id',
             'L1_time_in_hospital', 'L2_time_in_hospital', 'L3_time_in_hospital', 'L4_time_in_hospital', #y
             'L1_diagnosis', 'L2_diagnosis', 'L3_diagnosis',
             'L1_number_emergency', 'L2_number_emergency', 'L3_number_emergency']]

# Generate interval mappings
interval_mapping = generate_interval_mappings(df, interval_columns)

# Apply the interval mapping to create new categorical columns
for column, mapping in interval_mapping.items():
    df[mapping] = df[column].apply(lambda interval: mapping if interval in interval_mapping else None)

# Drop the original columns with intervals
for column in interval_mapping.keys():
    df.drop(column, axis=1, inplace=True)


df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74647 entries, 0 to 74646
Data columns (total 45 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   gender                    74647 non-null  object
 1   readmitted                74647 non-null  object
 2   discharge_disposition_id  74647 non-null  int64 
 3   admission_source_id       74647 non-null  int64 
 4   num_lab_procedures        74647 non-null  int64 
 5   number_outpatient         74647 non-null  int64 
 6   number_inpatient          74647 non-null  int64 
 7   number_diagnoses          74647 non-null  int64 
 8   max_glu_serum             74647 non-null  object
 9   A1Cresult                 74647 non-null  object
 10  metformin                 74647 non-null  object
 11  repaglinide               74647 non-null  object
 12  nateglinide               74647 non-null  object
 13  glimepiride               74647 non-null  object
 14  glipizide             

## K-anonymity and Simulated Annealing

### Quasi identiifers are formed as `'key': ['set1', 'set2',...]`. The aim to swap from each corresponding set later and not randomly from the whole columns.

In [ ]:
# Define the QI columns and their possible values
quasi_identifiers = {
    'age': ['L1_age', 'L2_age', 'L3_age'],
     'gender': ['gender'],
    'race': ['L1_race', 'L2_race', 'L3_race', 'L4_race'],
    'admission_type_id': [ 'L1_admission_type_id', 'L2_admission_type_id', 'L3_admission_type_id'],
    'time_in_hospital': ['L1_time_in_hospital', 'L2_time_in_hospital', 'L3_time_in_hospital', 'L4_time_in_hospital'],
    'diagnosis': ['L1_diagnosis', 'L2_diagnosis', 'L3_diagnosis'],
    'number_emergency': ['L1_number_emergency', 'L2_number_emergency', 'L3_number_emergency']
}




# Define the initial solution randomly
initial_solution = [
    random.choice(quasi_identifiers['age']),
    random.choice(quasi_identifiers['race']),
    random.choice(quasi_identifiers['admission_type_id']),
    random.choice(quasi_identifiers['time_in_hospital']),
    random.choice(quasi_identifiers['diagnosis']),
    random.choice(quasi_identifiers['number_emergency']),
    random.choice(quasi_identifiers['gender'])



]

initial_solution = ['L1_age', 'L4_race', 'L2_admission_type_id', 'L3_time_in_hospital', 'L2_diagnosis', 'L3_number_emergency', 'gender']
print(initial_solution)

['L1_age', 'L4_race', 'L2_admission_type_id', 'L3_time_in_hospital', 'L2_diagnosis', 'L3_number_emergency', 'gender']


###Simulated annealing parameters

In [ ]:
# temperature and cooling rate for SA
initial_temperature = 1000.0
cooling_rate = 0.01
min_temperature = 0.001

# number of iterations at each temperature level
num_iterations = 10 #200

# maximum number of iterations without improvement
max_iterations_without_improvement = 8

total_iterations = 300 #2000  # Initialize the total number of iterations

# Calculate the number of temperature levels based on total_iterations and num_iterations
num_temperature_levels = total_iterations // num_iterations

# Use num_temperature_levels to set the range of the temperature loop
current_temperature = initial_temperature

# Tried to implement adaptive cooling schedule but got confused more
'''
while current_temperature > min_temperature:
    total_iterations += num_iterations
    current_temperature *= cooling_rate

print("Max. Number of Iterations:", total_iterations)
'''
# Adjust the current temperature using the cooling rate
for _ in range(num_temperature_levels):
    current_temperature *= cooling_rate

### Classification model

In [ ]:
!pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 5.8 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score

def calculate_classification_accuracy(selected_columns, df, k):

    additional_columns = ['discharge_disposition_id', 'admission_source_id', 'num_lab_procedures', 'number_outpatient', 'number_inpatient', 'number_diagnoses', 'max_glu_serum', 'A1Cresult', 'metformin', 'repaglinide', 'nateglinide', 'glimepiride', 'glipizide', 'glyburide', 'pioglitazone', 'rosiglitazone', 'insulin', 'change', 'diabetesMed', 'service_utilization', 'numchange']

# Prepare the feature matrix (X) and target variable (y)
    X = df[selected_columns + additional_columns]
    y = df['readmitted']

    # Print the number of rows in X and y
    print('X columns:', X.columns)
    print("Number of rows in X:", len(X))
    print("Number of rows in y:", len(y))

    # Convert categorical features to strings
    categorical_features_indices = X.select_dtypes(include=['object']).columns

    random_seed = 32
    np.random.seed(random_seed)

    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=random_seed)

    # Initialize the CatBoostClassifier
    catboost_model = CatBoostClassifier(iterations=100, depth=10, learning_rate=0.1, loss_function='Logloss', cat_features=list(categorical_features_indices))

    # Train the model on the training data
    catboost_model.fit(X_train, y_train, cat_features=list(categorical_features_indices))

    # Convert true labels to numeric format
    y_test = y_test.astype(int)

    # Predict on the testing data
    y_pred = catboost_model.predict(X_test)
    y_pred = y_pred.astype(int)
    # Print some information for debugging
    print("Unique values in y:", y.unique())
    print("Number of unique predictions:", len(set(y_pred)))

    # Calculate and return the classification accuracy
    accuracy = accuracy_score(y_test, y_pred)


    # Calculate the AUC-ROC score
    auc_roc = roc_auc_score(y_test, y_pred)
    print("AUC-ROC Score:", auc_roc)

    return auc_roc


### Objective Function

In [ ]:
# k-anonymity threshold
k = 4

def objective_function(solution, max_suppressed_fraction):
    # Group the data based on the solution
    grouped = df.groupby(solution)
    total_rows_count = 0
    '''
    for group_name, group_data in grouped:
        #group_sum = group_data.sum()
        #print(f"Group Name: {group_name}")
        #print(f"Sum of Rows in Group:\n{group_sum}")
        group_rows_count = len(group_data)
        total_rows_count += group_rows_count

        # Print the total sum of all rows in all groups (just to test that nothing changed)
    print(f"Total Sum of All Rows in All Groups: {total_rows_count}")
    '''

    #print('Grouped rows', grouped)
    #print('SOLUTION', solution)
    #df_tmp = df
    #df_tmp = df[solution]
    additional_columns = [ 'discharge_disposition_id', 'admission_source_id', 'num_lab_procedures', 'number_outpatient', 'number_inpatient', 'number_diagnoses',
             'max_glu_serum', 'A1Cresult', 'metformin', 'repaglinide', 'nateglinide', 'glimepiride', 'glipizide', 'glyburide', 'pioglitazone', 'rosiglitazone', 'insulin', 'change',
             'diabetesMed', 'service_utilization', 'numchange', 'num_procedures', 'num_medications', 'readmitted']
    df_tmp = df[solution + additional_columns]
    # print('DF TMP INFO', df_tmp)

    # Calculate the percentage of suppressed rows -- utility
    total_rows = len(df)
    suppressed_rows = sum(len(group) for group_name, group in grouped if len(group) < k)
    suppressed_fraction = suppressed_rows / total_rows
    print('SUPPRESSION FRACTION', suppressed_fraction)
    #print('Total Suppressed Rows:', suppressed_rows)
    print('Max Suppressed Fraction:', max_suppressed_fraction)


    suppressed_fraction = round(suppressed_fraction, 2)
    if suppressed_fraction <= max_suppressed_fraction + 1e-6:
    #if testx < testy :

        # Filter the original dataframe to remove suppressed rows
        filtered_df = df_tmp[~df_tmp.index.isin([idx for group_name, group in grouped if len(group) < k for idx in group.index])]
        #print('FILTERED DF INFO', filtered_df.info())

        # Calculate and return classification accuracy as the objective function score
        classification_accuracy = calculate_classification_accuracy(solution, filtered_df, k)
        print('Classification accuracy (AUC-ROC)', classification_accuracy)
        return classification_accuracy

    else: #suppressed_fraction > max_suppressed_fraction:
        # If the constraint is violated, return a penalty (negative infinity)
        print('SUPPRESSION EXCEEDED LIMIT')
        return -float('inf')



###Neighbourhood solution

In [ ]:

def get_neighbor_solution(solution):
    neighbor_solution = solution.copy()  # Create a copy of the current solution

    # Define the column sets for swapping
    swap_sets = {
        #'sex': ['sex'],
        'age': ['L1_age', 'L2_age', 'L3_age'],
        'race': ['L1_race', 'L2_race', 'L3_race', 'L4_race'],
        'admission_type_id': [ 'L1_admission_type_id', 'L2_admission_type_id', 'L3_admission_type_id'],
        'time_in_hospital': ['L1_time_in_hospital', 'L2_time_in_hospital', 'L3_time_in_hospital', 'L4_time_in_hospital'],
        'diagnosis': ['L1_diagnosis', 'L2_diagnosis', 'L3_diagnosis'],
        'number_emergency': ['L1_number_emergency', 'L2_number_emergency', 'L3_number_emergency']
    }

    # Iterate over each column set and perform swapping if the current column is from that set
    # Randomly select a set to swap columns
    selected_set = random.choice(list(swap_sets.keys()))
    print('selected set:', selected_set)

    # Randomly shuffle the columns within the selected set
    random.shuffle(swap_sets[selected_set])

    # Replace the corresponding columns in the current solution with the shuffled columns
    for idx, col in enumerate(solution):
        if col in swap_sets[selected_set]:
            neighbor_solution[idx] = random.choice(swap_sets[selected_set])
            print('current solution:', solution)
            print('neighbor solution:', neighbor_solution)

    return neighbor_solution

###Simulated Annealing

In [ ]:
# Print unique values in the 'readmitted' column
print("Unique values in 'readmitted' column:", df['readmitted'].unique())


Unique values in 'readmitted' column: ['1' '0']


In [ ]:
import math
import random

def simulated_annealing(initial_solution, initial_temperature, cooling_rate, min_temperature, max_iterations_without_improvement, max_suppressed_fraction):
    current_solution = initial_solution  # Store the current solution
    best_solution = initial_solution  # Initialize the best solution as the current solution
    current_temperature = initial_temperature  # Start with the initial temperature
    iterations_without_improvement = 0  # Initialize counter for iterations without improvement
    iteration = 0  # Initialize the iteration counter


    while current_temperature > min_temperature and iterations_without_improvement < max_iterations_without_improvement:
        improved = False

        for _ in range(num_iterations):
            iteration += 1
            print(f"Iteration {iteration}")

            neighbor_solution = get_neighbor_solution(current_solution)

            current_cost = objective_function(current_solution, max_suppressed_fraction)
            print('current_cost:', current_cost)
            neighbor_cost = objective_function(neighbor_solution, max_suppressed_fraction)
            print('neighbor_cost:', neighbor_cost)


            if neighbor_cost >= current_cost:
                current_solution = neighbor_solution
                #if neighbor_cost < objective_function(best_solution, max_suppressed_fraction):
                if neighbor_cost > objective_function(best_solution, max_suppressed_fraction):# and current_cost <= max_suppressed_fraction :
                    best_solution = neighbor_solution
                    print('current best solution:', best_solution)
                    improved = True
            else:
                acceptance_probability = math.exp((current_cost - neighbor_cost) / current_temperature)
                if random.random() < acceptance_probability:
                    current_solution = neighbor_solution
                    improved = True

            print('CURRENT SOLUTION', current_solution)
            if iteration % num_iterations == 0:
                current_temperature *= cooling_rate
                iterations_without_improvement = 0

            if not improved:
                iterations_without_improvement += num_iterations

        current_temperature *= cooling_rate
        iterations_without_improvement += num_iterations

        if improved:
            iterations_without_improvement = 0
        else:
            iterations_without_improvement += num_iterations

    return best_solution


max_suppressed_fraction = 0.10
best_solution = simulated_annealing(initial_solution, initial_temperature, cooling_rate, min_temperature, max_iterations_without_improvement, max_suppressed_fraction)
#max_suppressed_fraction = 0.50  # User-defined maximum suppressed fraction of rows
#auc_roc = objective_function(best_solution, max_suppressed_fraction)  # Calculate the classification accuracy for the best solution



print("Best Solution:", best_solution)
#print("AUC-ROC score:", auc_roc)


Streaming output truncated to the last 5000 lines.
11:	learn: 0.4962018	total: 1.74s	remaining: 12.7s
12:	learn: 0.4839781	total: 1.88s	remaining: 12.6s
13:	learn: 0.4790994	total: 2.03s	remaining: 12.5s
14:	learn: 0.4715515	total: 2.17s	remaining: 12.3s
15:	learn: 0.4670490	total: 2.3s	remaining: 12.1s
16:	learn: 0.4625643	total: 2.44s	remaining: 11.9s
17:	learn: 0.4590345	total: 2.6s	remaining: 11.8s
18:	learn: 0.4513494	total: 2.74s	remaining: 11.7s
19:	learn: 0.4446975	total: 2.87s	remaining: 11.5s
20:	learn: 0.4409501	total: 3.02s	remaining: 11.4s
21:	learn: 0.4366534	total: 3.16s	remaining: 11.2s
22:	learn: 0.4332547	total: 3.3s	remaining: 11s
23:	learn: 0.4304058	total: 3.44s	remaining: 10.9s
24:	learn: 0.4267777	total: 3.6s	remaining: 10.8s
25:	learn: 0.4234212	total: 3.73s	remaining: 10.6s
26:	learn: 0.4208590	total: 3.87s	remaining: 10.5s
27:	learn: 0.4184269	total: 4.01s	remaining: 10.3s
28:	learn: 0.4159284	total: 4.15s	remaining: 10.2s
29:	learn: 0.4137583	total: 4.29s	rem

### Testing the correctness of K anonymity on the best_solution (near-optimal combination of generalized attributes) produced by the SA algorithm

In [ ]:

# Define the privacy parameter
#k = 3

#For testing purposes
#test_solution = ['L1_age', 'L3_race', 'L2_admission_type_id', 'L3_time_in_hospital', 'L2_diagnosis', 'L3_number_emergency', 'gender']

test_solution = best_solution

df_k = df

# Group the dataset by the quasi-identifiers and suppress the groups with less than k rows
grouped = df_k.groupby(test_solution)
suppressed_indices = []
for group_name, group in grouped:
    if len(group) < k:
        suppressed_indices.extend(group.index)
df_k = df_k.drop(suppressed_indices)
#df_k.info()
# Restore the original index
df_index = df_k.index
df_k = df_k.reset_index(drop=True)

counter = 0
for index in suppressed_indices:
    counter = counter + 1
    #print(df_k.loc[index])
    #print("------------")
print('sum of supressed records:', counter)
total_rows = len(df)
suppressed_rows = counter
suppressed_fraction = suppressed_rows / total_rows
print('SUPPRESSION FRACTION', suppressed_fraction)


sum of supressed records: 2465
SUPPRESSION FRACTION 0.033022090639945344


### Run the classification model again but with 500 iterations to find the best classification accuracy the attribute set can achieve

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score, classification_report

selected_columns = best_solution
#k=5

df_k = df
#org

additional_columns = ['discharge_disposition_id', 'admission_source_id', 'num_lab_procedures', 'number_outpatient', 'number_inpatient', 'number_diagnoses',
             'max_glu_serum', 'A1Cresult', 'metformin', 'repaglinide', 'nateglinide', 'glimepiride', 'glipizide', 'glyburide', 'pioglitazone', 'rosiglitazone', 'insulin', 'change',
             'diabetesMed', 'service_utilization', 'numchange', 'num_medications', 'num_procedures', 'readmitted']

#features = ['L1_race', 'gender', 'L3_age', 'L3_admission_type_id', 'L1_time_in_hospital', 'L1_number_emergency', 'L3_diagnosis']
features = best_solution

# Group the dataset by the quasi-identifiers and suppress the groups with less than k rows
grouped = df_k.groupby(features)
suppressed_indices = []
for group_name, group in grouped:
    if len(group) < k:
        suppressed_indices.extend(group.index)
df_k = df_k.drop(suppressed_indices)
df_k.info()
# Restore the original index
df_index = df_k.index
df_k = df_k.reset_index(drop=True)



featurez_matrix = df_k[features + additional_columns]


# Prepare the feature matrix (X) and target variable (y)
X = featurez_matrix.drop('readmitted', axis=1)
y = featurez_matrix['readmitted']


# Convert categorical features to strings
categorical_features_indices = X.select_dtypes(include=['object']).columns
#X[categorical_features_indices] = X[categorical_features_indices].astype(str)


counter = 0
for index in suppressed_indices:
    counter = counter + 1
    #print(df_k.loc[index])
    #print("------------")
print('sum of supressed records:', counter)
total_rows = len(df)
suppressed_rows = counter
suppressed_fraction = suppressed_rows / total_rows
print('SUPPRESSION FRACTION', suppressed_fraction)
print('K-value:', k)
print('anon df columns:', df_k.columns)


# Convert categorical features to strings
categorical_features_indices = X.select_dtypes(include=['object']).columns
#X[categorical_features_indices] = X[categorical_features_indices].astype(str)
#seed = 23 for tests, seed= 28 for baseline
random_seed = 28
np.random.seed(random_seed)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=random_seed)

# Initialize the CatBoostClassifier
catboost_model = CatBoostClassifier(iterations=500, depth=10, learning_rate=0.2, loss_function='Logloss', cat_features=list(categorical_features_indices))

# Train the model on the training data
catboost_model.fit(X_train, y_train, cat_features=list(categorical_features_indices))  # Convert to list here

# Convert true labels to numeric format
y_test = y_test.astype(int)

# Predict on the testing data
y_pred = catboost_model.predict(X_test)
y_pred = y_pred.astype(int)


# Evaluate the model's performance
classification_metrics = classification_report(y_test, y_pred, digits=3)
print(classification_metrics)


# Calculate the AUC-ROC score
auc_roc = roc_auc_score(y_test, y_pred)
print("AUC-ROC Score: {:.3f}".format(auc_roc))


counter = 0
for index in suppressed_indices:
    counter = counter + 1
    #print(df_k.loc[index])
    #print("------------")
print('sum of supressed records:', counter)
total_rows = len(df)
suppressed_rows = counter
suppressed_fraction = suppressed_rows / total_rows
print('SUPPRESSION FRACTION', suppressed_fraction)
print('K-value:', k)
print('anon df columns:', X.columns)
print('best solution', best_solution)

In [ ]:
X.columns

Index(['L3_age', 'L1_race', 'L1_admission_type_id', 'L3_time_in_hospital',
       'L2_diagnosis', 'L2_number_emergency', 'gender',
       'discharge_disposition_id', 'admission_source_id', 'num_lab_procedures',
       'number_outpatient', 'number_inpatient', 'number_diagnoses',
       'max_glu_serum', 'A1Cresult', 'metformin', 'repaglinide', 'nateglinide',
       'glimepiride', 'glipizide', 'glyburide', 'pioglitazone',
       'rosiglitazone', 'insulin', 'change', 'diabetesMed',
       'service_utilization', 'numchange', 'num_medications',
       'num_procedures'],
      dtype='object')

In [ ]:
stohere

NameError: name 'stohere' is not defined

Run this again to check if the number of supressed rows outputted by the simulated annealing is actually the optimal one. The snipped takes the set of features returned the the SA algorithm and performs k-anonymity on them.

test k-anon after sa

## Classification


In [ ]:
!pip install catboost

Baseline

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier
from sklearn.impute import SimpleImputer
from imblearn.over_sampling import SMOTE
from sklearn.metrics import accuracy_score, classification_report

selected_columns = best_solution

# Additional columns to include in the feature set
# maybe add oasis and glucose later
'''
features = ['sex', 'L7_age', 'L2_race', 'L3_marital_status', 'L5_education',
       'L3_native_country', 'L3_workclass', 'L3_occupation', 'readmitted',
       'L1_age', 'L2_age', 'L3_age', 'L4_age', 'L5_age', 'L6_age',
       'L1_education', 'L2_education', 'L3_education', 'L4_education',
       'L1_occupation', 'L2_occupation', 'L1_marital_status',
       'L2_marital_status', 'L1_workclass', 'L2_workclass',
       'L1_native_country', 'L2_native_country', 'L1_race']

'''

features = ['L4_age', 'L3_race', 'L1_admission_type_id', 'L2_time_in_hospital', 'L3_num_procedures', 'L1_num_medications', 'L3_diagnosis', 'L2_number_emergency', 'readmitted']



featurez_matrix = df[features]


# Prepare the feature matrix (X) and target variable (y)
X = featurez_matrix.drop('readmitted', axis=1)
y = featurez_matrix['readmitted']


# Convert categorical features to strings
categorical_features_indices = X.select_dtypes(include=['object']).columns
#X[categorical_features_indices] = X[categorical_features_indices].astype(str)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)



# Initialize the CatBoostClassifier
catboost_model = CatBoostClassifier(iterations=700, depth=10, learning_rate=0.1, loss_function='Logloss', cat_features=list(categorical_features_indices))

# Train the model on the training data
catboost_model.fit(X_train, y_train, cat_features=list(categorical_features_indices))  # Convert to list here

# Predict on the testing data
y_pred = catboost_model.predict(X_test)
# Convert the predicted labels to string type
y_pred = y_pred.astype(str)


# Evaluate the model's performance
classification_metrics = classification_report(y_test, y_pred)
print(classification_metrics)

# Calculate AUC-ROC score

auc_roc = roc_auc_score(y_test.astype(int), y_pred.astype(int))
print("AUC-ROC Score:", auc_roc)



### Archived Code

In [ ]:
#df[selected_columns]

In [ ]:
'''
import pandas as pd
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier
from sklearn.impute import SimpleImputer
from imblearn.over_sampling import SMOTE
from sklearn.metrics import accuracy_score, classification_report

selected_columns = best_solution


feature_matrix = df[selected_columns]




#########################

##ALTERNATIVE K-ANONYM:
#df_test = df.copy()

# Group the dataset by the quasi-identifiers and suppress the groups with less than k rows
grouped = feature_matrix.groupby(selected_columns)

suppressed_indices = []
for group_name, group in grouped:
    if len(group) < k:
        suppressed_indices.extend(group.index)
       #print(f"Group: {group_name}")
        #print(f"Number of Rows: {len(group)}")
        #print(f"Columns:")
        #print(group)
        #print("--------------------")


feature_matrix = feature_matrix.drop(suppressed_indices)
#df.info()

# Restore the original index
#df_index = df_test.index
#df_test = df_test.reset_index(drop=True)

#counter = 0

#for index in suppressed_indices:
#    counter = counter + 1
    #print(df.loc[index])
    #print("------------")

print("Suppressed Indices:", suppressed_indices)


########END#############



# Prepare the feature matrix (X) and target variable (y)
X = df[selected_columns]
y = df['salary-class']


# Convert categorical features to strings
categorical_features_indices = X.select_dtypes(include=['object']).columns
#X[categorical_features_indices] = X[categorical_features_indices].astype(str)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)



# Initialize the CatBoostClassifier
catboost_model = CatBoostClassifier(iterations=500, depth=10, learning_rate=0.1, loss_function='Logloss', cat_features=list(categorical_features_indices))

# Train the model on the training data
catboost_model.fit(X_train, y_train, cat_features=list(categorical_features_indices))  # Convert to list here

# Predict on the testing data
y_pred = catboost_model.predict(X_test)
# Convert the predicted labels to string type
y_pred = y_pred.astype(str)


# Evaluate the model's performance
classification_metrics = classification_report(y_test, y_pred)
print(classification_metrics)

# Calculate AUC-ROC score
#auc_roc = roc_auc_score(y_test, y_pred)
#print("AUC-ROC Score:", auc_roc)
#print('sum of supressed records:', len(suppressed_indices))
# Calculate AUC-ROC score
auc_roc = roc_auc_score(y_test.astype(int), y_pred.astype(int))
print("AUC-ROC Score:", auc_roc)
print('sum of suppressed records:', len(suppressed_indices))

'''

In [ ]:
#print("Categorical Feature Indices:", categorical_features_indices)
#print("Best Solution:", best_solution)
#print("Optimization score:", suppressed_rows)

In [ ]:
'''
import matplotlib.pyplot as plt

# Get feature importances
feature_importances = catboost_model.feature_importances_

# Get feature names
feature_names = X.columns

# Sort feature importances in descending order
sorted_idx = np.argsort(feature_importances)[::-1]

# Plot feature importances
plt.figure(figsize=(20, 6))
plt.bar(range(len(feature_names)), feature_importances[sorted_idx])
plt.xticks(range(len(feature_names)), np.array(feature_names)[sorted_idx], rotation='vertical')
plt.xlabel("Feature")
plt.ylabel("Feature Importance")
plt.title("Feature Importances")


# Annotate feature importances next to the bars
for i, idx in enumerate(sorted_idx):
    plt.text(i, feature_importances[idx] + 0.01, f"{feature_importances[idx]:.3f}", ha='center')

plt.tight_layout()
plt.show()
'''

In [ ]:
#X.info()